<a href="https://colab.research.google.com/github/sowmyarajesh/ML_ANN/blob/main/ANN_BinaryClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)

2.8.0


### Load and preprocess the data

In [4]:
# import dataset from the local 
from google.colab import files
upload  = files.upload()

Saving gender_submission.csv to gender_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [5]:
!ls

gender_submission.csv  sample_data  test.csv  train.csv


In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

understanding the data

In [7]:
# size of the training and testing data set
train.shape, test.shape


((891, 12), (418, 11))

In [9]:
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


9

### Build the model

To build the model,first initialize the model


In [32]:
# initialize the sequence
model = tf.keras.models.Sequential()


**Input layer**

 we will use the following parameters in input layer
(units=128,  activation='relu',  input_shape=(784,))


units : This can be any number with in the number of inputs to be filters from the input features. We start with different 128. 

activation: relu is the activation function as it is faster than sigmoid/ tanh functions

input_shape : it shows the number of columns in the input. 

In [33]:
inputLayer = tf.keras.layers.Dense(units=255, activation='relu', input_shape=((784,)))
model.add(inputLayer)

**Hidden Layer:**

Add a hidden layer for dropout (regulaization technique where the features are randomly ignored during the training process. This means the ignored features are not considered during the backward propagation process).

Dropout layer will help in reducing the chances of overfitting the model

Dropout function will take the percentage of the features to drop

In [34]:
hiddenLayer1 = tf.keras.layers.Dropout(0.2)
model.add(hiddenLayer1)

**Output layer:**

Only two parameters are needed for the output layer in this function

number of units to output = 10 as we have 10 classes

activation later = softmax for this model.

softmax function is used for multiclass classification

sigmoid function is used for binary classification

In [35]:
outputLayer = tf.keras.layers.Dense(units=10, activation="softmax")
model.add(outputLayer)

### Train the model

**Compile the model:**

First step to training is compile the defined model. 

The compiling step requires three parameters

optimizer = the function used to minimize the loss function. "adam" is the most common optimizer used

loss function = acts as a guide to the optimizer in the right direction so that optimizer can reach the global minimum. Since this is a multi-label classification problem, we will be using sparse_categorical_crossentropy

metrices = this will list the performance metrices to be considered in guiding the optimizer in the right direction. In this model, we will be using the sparse_categorical_accuracy

In [51]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [ ]:
model.summary()

In [ ]:
model.fit(x=x_train, y=y_train, epochs=10)

### Evaluate the model

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test,y_test)

In [55]:
print("Accuracy value = ", test_accuracy)

Accuracy value =  0.8903999924659729


In [ ]:
from sklearn import metrics

y_pred =np.argmax(model.predict(x_test),axis=1)
print(y_pred)
metrics.confusion_matrix(y_test,y_pred)

In [ ]:
metrics.accuracy_score(y_test,y_pred)